This is an attempt to partially replicate the model described in ["The Evolution of Layered Protocol Stacks Leads to an Hourglass-Shaped Architecture" (Saamer Akshabi, Constantine Dovrolis)](https://www.cc.gatech.edu/~dovrolis/Papers/evoarch.pdf), for fun.

In [3]:
import random
random.seed(1000)

In [1]:
from typing import List

def default_generalities(layers: int = 10) -> List[float]:
    return [(layers - i - 1) / layers for i in range(layers)]

print(f'Default generality vector: {default_generalities()}')

Default generality vector: [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0]


In [2]:
from dataclasses import dataclass, field

@dataclass
class Params:
    generalities: List[float] = field(default_factory=default_generalities)  # aka s(l)
    competition_threshold: float = 0.6  # aka c
    init_nodes_per_layer: int = 10
    birth_rate: float = 0.05
    mortality: float = 1.0  # aka z
    min_rounds: int = 100
    max_nodes: int = 500

print(f'Default model params: {Params()}')

Default model params: Params(generalities=[0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0], competition_threshold=0.6, init_nodes_per_layer=10, birth_rate=0.05, mortality=1.0, min_rounds=100, max_nodes=500)


In [7]:
from __future__ import annotations
from typing import Dict, Set

@dataclass(eq=False)
class Node:
    product_nids: Set[int] = field(default_factory=set)
    value: int = 0

@dataclass(eq=False)
class Layer:
    nodes: Dict[int, Node] = field(default_factory=dict)

@dataclass(eq=False)
class Arch:
    params: Params
    rounds_done: int = 0
    layers: List[Layer] = field(default_factory=list)
    prev_nid: int = 0
    
    def next_nid(self) -> int:
        self.prev_nid += 1
        return self.prev_nid

def assign_substrates(arch: Arch, lid: int, nid: int) -> None:
    if lid == 0:
        return
    generality = arch.params.generalities[lid-1]
    for candidate_nid, candidate in arch.layers[lid-1].nodes.items():
        if random.random() < generality:
            candidate.product_nids.add(nid)

def update_value(arch: Arch, lid: int, nid: int) -> None:
    node = arch.layers[lid].nodes[nid]
    if lid == len(arch.layers) - 1:
        node.value = 1
    else:
        node.value = sum(arch.layers[lid+1].nodes[p].value for p in node.product_nids)

def create_arch(params):
    arch = Arch(params)
    nlayers = len(params.generalities)
    
    # Create initial nodes
    for i in range(nlayers):
        layer = Layer()
        for j in range(params.init_nodes_per_layer):
            layer.nodes[arch.next_nid()] = Node()
        arch.layers.append(layer)
    
    # Choose substrates for initial nodes and calculate values
    for lid in reversed(range(nlayers)):
        for nid in arch.layers[lid].nodes:
            update_value(arch, lid, nid)
            assign_substrates(arch, lid, nid)
    
    return arch

print(f'Default model: {create_arch(Params())}')

Default model: Arch(params=Params(generalities=[0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0], competition_threshold=0.6, init_nodes_per_layer=10, birth_rate=0.05, mortality=1.0, min_rounds=100, max_nodes=500), rounds_done=0, layers=[Layer(nodes={1: Node(product_nids={11, 12, 13, 14, 15, 16, 17, 18, 19, 20}, value=495079), 2: Node(product_nids={11, 12, 13, 14, 15, 17, 18, 19, 20}, value=452866), 3: Node(product_nids={12, 13, 14, 15, 17, 18, 19, 20}, value=410491), 4: Node(product_nids={11, 12, 13, 15, 16, 18, 19, 20}, value=395698), 5: Node(product_nids={12, 13, 14, 15, 16, 17, 18, 19, 20}, value=452704), 6: Node(product_nids={11, 14, 15, 16, 17, 19}, value=302572), 7: Node(product_nids={11, 12, 13, 14, 15, 16, 17, 18, 19, 20}, value=495079), 8: Node(product_nids={11, 12, 13, 14, 15, 16, 17, 18, 20}, value=432831), 9: Node(product_nids={11, 12, 13, 14, 16, 17, 18, 19, 20}, value=438724), 10: Node(product_nids={11, 13, 14, 15, 16, 17, 18, 19, 20}, value=450025)}), Layer(nodes={11: N